In [1]:
%cd ../src

/homes/hh2721/fyp/haashim_repo/src


In [4]:
import numpy as np

from util.privacy import PrivacyBudget

In [5]:
def __total_delta_safe(spent_budget, slack):
        delta_spend = [slack]
        for _, delta in spent_budget:
            delta_spend.append(delta)
        delta_spend.sort()

        # (1 - a) * (1 - b) = 1 - (a + b - a * b)
        prod = 0
        for delta in delta_spend:
            prod += delta - prod * delta

        return prod

In [6]:
def total(spent_budget=None, slack=None):
    epsilon_sum, epsilon_exp_sum, epsilon_sq_sum = 0, 0, 0

    for epsilon, _ in spent_budget:
        epsilon_sum += epsilon
        epsilon_exp_sum += (1 - np.exp(-epsilon)) * epsilon / (1 + np.exp(-epsilon))
        epsilon_sq_sum += epsilon ** 2

    total_epsilon_naive = epsilon_sum
    total_delta = __total_delta_safe(spent_budget, slack)

    if slack == 0:
        return PrivacyBudget(total_epsilon_naive, total_delta)

    total_epsilon_drv = epsilon_exp_sum + np.sqrt(2 * epsilon_sq_sum * np.log(1 / slack))
    total_epsilon_kov = epsilon_exp_sum + np.sqrt(2 * epsilon_sq_sum *
                                                    np.log(np.exp(1) + np.sqrt(epsilon_sq_sum) / slack))

    return PrivacyBudget(min(total_epsilon_naive, total_epsilon_drv, total_epsilon_kov), total_delta)

In [10]:
def split_privacy_budget(k: int, budget: PrivacyBudget) -> PrivacyBudget:
    delta = 1 - ((1 - budget.delta) / (1)) ** (1 / k)

    lower = 0
    upper = budget.epsilon
    old_interval_size = (upper - lower) * 2

    while old_interval_size > upper - lower:
        old_interval_size = upper - lower
        mid = (upper + lower) / 2

        spent_budget = [(mid, 0)] * k
        cost = total(spent_budget=spent_budget, slack=budget.delta)

        if cost.epsilon >= budget.epsilon:
            upper = mid
        if cost.epsilon <= budget.epsilon:
            lower = mid

    epsilon = (upper + lower) / 2

    return PrivacyBudget(epsilon, delta)

In [14]:
split_privacy_budget(10, PrivacyBudget(5.0, 1e-5))

PrivacyBudget(epsilon=0.5, delta=1.000004499984719e-06)